## Goal:  determine which of three marketing promotions is most effective for a new menu item at a fast-food chain, measured across multiple locations over four weeks.

The key components we're working with are:

* Three different promotions (our test variants)
* Multiple store locations
* Four weeks of sales data
* Different market sizes and store ages

Let's load and examine our data.

In [7]:
import sys
import pandas as pd

sys.path.append('..')

import kagglehub

from src.analysis import ABTestAnalysis
from src.visualization import VisualizationManager

path = kagglehub.dataset_download("chebotinaa/fast-food-marketing-campaign-ab-test")
df = pd.read_csv(path + "/WA_Marketing-Campaign.csv")

display(df.head())
display(df.info())

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MarketID          548 non-null    int64  
 1   MarketSize        548 non-null    object 
 2   LocationID        548 non-null    int64  
 3   AgeOfStore        548 non-null    int64  
 4   Promotion         548 non-null    int64  
 5   week              548 non-null    int64  
 6   SalesInThousands  548 non-null    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 30.1+ KB


None

When analyzing sales data, we need to consider a few important factors that could affect our interpretation:

* Raw sales vs. normalized sales: Since we have different store sizes and market conditions, we might want to consider looking at the percentage increase in sales rather than just the raw numbers. This helps account for the fact that larger stores naturally have higher sales volumes.
* Time component: We have data for four weeks, which means we can look at how the effect of the promotion changes over time.

Let's start by creating a baseline analysis of our sales data:

In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate average sales by promotion
promotion_summary = df.groupby('Promotion')['SalesInThousands'].agg([
    'mean',
    'std',
    'count'
]).round(2)

# Create a box plot using Plotly Express
fig_box = px.box(df,
                 x='Promotion',
                 y='SalesInThousands',
                 title='Sales Distribution by Promotion Type',
                 labels={'SalesInThousands': 'Sales (Thousands)',
                         'Promotion': 'Promotion Type'})
fig_box.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=800,
    height=500
)
fig_box.show()

# Create line plot for weekly trends
weekly_sales = df.groupby(['week', 'Promotion'])['SalesInThousands'].mean().reset_index()
fig_line = px.line(weekly_sales,
                   x='week',
                   y='SalesInThousands',
                   color='Promotion',
                   markers=True,
                   title='Average Sales by Promotion Over Time',
                   labels={'SalesInThousands': 'Average Sales (Thousands)',
                           'week': 'Week',
                           'Promotion': 'Promotion Type'})
fig_line.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=800,
    height=500,
    hovermode='x unified'
)



The box itself represents where 50% of all sales fall (called the interquartile range or IQR)
The line in the middle of the box is the median sale value
The whiskers show us the spread of the remaining data (excluding outliers)
The individual points beyond the whiskers are outliers

This box plot shows us that:

Looking at Promotion 1, we can see that:

* It has a higher median (the middle line in the box) compared to Promotions 2 and 3
* The box (middle 50% of sales) sits higher on the y-axis
* It has several high-performing outliers near 100,000 in sales

* Variability: Notice how the box for Promotion 1 is taller than Promotion 2? This means there's more variability in the sales results. Sometimes variability can be risky for business - would you rather have:
    * Consistent sales around $50,000
    * Sales that swing between $30,000 and $70,000

Market Conditions: We should consider whether these promotions were tested in comparable conditions. For example, were they all tested in similar-sized markets?

In [9]:
fig_line.show()

Now, looking at the time series (line) graph, we can see patterns:

* Promotion 1 (blue line) shows stability at a higher level, with slight fluctuations
* Promotion 2 (red line) is consistently lower and shows a concerning downward trend in the final week
* Promotion 3 (green line) stays in the middle and shows some recovery in the final week after a dip

This time-based view adds context to our earlier observation. While Promotion 1 does have higher variability in individual store performance (as shown in the box plot), its average performance over time is actually quite stable. This suggests that the variability we're seeing might be more related to store-specific factors (like location or store size) rather than the promotion itself becoming less effective over time.

## Sample Ratio Mismatch (SRM) Test

Let's check for sample ratio mismatch (SRM) using a chi-square test. This is like checking if we've dealt our cards fairly before starting a game - we want to make sure our test groups are reasonably balanced.

In [10]:
import pandas as pd
import numpy as np
from scipy import stats

# Count samples in each promotion group
sample_counts = df.groupby('Promotion')['LocationID'].nunique()

# Perform chi-square test for equal proportions
total_samples = sample_counts.sum()
expected = total_samples / 3  # Expected count if perfectly balanced
chi_stat, p_value = stats.chisquare(sample_counts, [expected] * 3)

print("Sample sizes per promotion:")
print(sample_counts)
print(f"\nChi-square test results:")
print(f"Chi-square statistic: {chi_stat:.2f}")
print(f"p-value: {p_value:.4f}")

Sample sizes per promotion:
Promotion
1    43
2    47
3    47
Name: LocationID, dtype: int64

Chi-square test results:
Chi-square statistic: 0.23
p-value: 0.8898


The chi-square test for SRM helps us verify if our test was set up fairly. If the p-value is less than 0.05, we might have a problem with how the promotions were distributed.

Now, for our main analysis, we'll use a one-way ANOVA followed by pairwise t-tests. Think of ANOVA as asking "Are there any differences between these groups?" while t-tests help us pinpoint exactly which groups are different from each other.

In [11]:
# One-way ANOVA
promotions = [group['SalesInThousands'].values for name, group in df.groupby('Promotion')]
f_stat, anova_p = stats.f_oneway(*promotions)

# Pairwise t-tests
pairwise_tests = []
promotions_data = {i: df[df['Promotion'] == i]['SalesInThousands'] for i in [1, 2, 3]}

for i in [1, 2, 3]:
    for j in range(i + 1, 4):
        t_stat, t_p = stats.ttest_ind(promotions_data[i], promotions_data[j])
        effect_size = promotions_data[i].mean() - promotions_data[j].mean()
        # Calculate confidence interval analytically
        n1, n2 = len(promotions_data[i]), len(promotions_data[j])
        pooled_se = np.sqrt(promotions_data[i].var() / n1 + promotions_data[j].var() / n2)
        ci = stats.t.interval(0.95, n1 + n2 - 2, loc=effect_size, scale=pooled_se)

        pairwise_tests.append({
            'Comparison': f'Promotion {i} vs {j}',
            'Effect Size': effect_size,
            'P-value': t_p,
            'CI_lower': ci[0],
            'CI_upper': ci[1]
        })

# Create a DataFrame with results
results_df = pd.DataFrame(pairwise_tests)
display(results_df)


# Let's also calculate bootstrap confidence intervals for Promotion 1 vs 2
def bootstrap_ci(data1, data2, n_bootstrap=10000):
    differences = []
    for _ in range(n_bootstrap):
        sample1 = np.random.choice(data1, size=len(data1), replace=True)
        sample2 = np.random.choice(data2, size=len(data2), replace=True)
        differences.append(np.mean(sample1) - np.mean(sample2))

    return np.percentile(differences, [2.5, 97.5])


boot_ci = bootstrap_ci(promotions_data[1], promotions_data[2])
print(f"\nBootstrap CI for Promotion 1 vs 2: {boot_ci}")
boot_ci

,Comparison,Effect Size,P-value,CI_lower,CI_upper
0,Promotion 1 vs 2,10.769597,3.550670e-10,7.474454,14.064739
1,Promotion 1 vs 3,2.734544,1.207967e-01,-0.721568,6.190655
2,Promotion 2 vs 3,-8.035053,1.562894e-06,-11.271741,-4.798366



Bootstrap CI for Promotion 1 vs 2: [ 7.57512853 14.03415905]


array([ 7.57512853, 14.03415905])

The ANOVA test tells us if there are any significant differences between the promotions. Think of it as asking "Is there anything interesting to look at here?"

The pairwise t-tests compare each promotion against the others. For each comparison, we get:

* The effect size (how much better one promotion is than another)
* A p-value (how confident we can be that the difference is real)
* A confidence interval (the range where we think the true difference lies)

We calculated confidence intervals in two ways for Promotion 1 vs 2:

* Analytically (using mathematical formulas)
* Using bootstrap (by resampling our data many times)

# Analysis of Promotional Results

### Sample Ratio Mismatch (SRM) Test
* Distribution shows 43 locations for Promotion 1 and 47 locations each for Promotions 2 and 3
* Chi-square statistic: 0.23, p-value: 0.8898 indicates fair sample distribution

### Pairwise Comparisons

**Promotion 1 vs. Promotion 2:**
* Effect Size: +10.77k sales (favoring Promotion 1)
* P-value: 3.56e-10
* Confidence Intervals:
 * Standard: [7.47k, 14.86k]
 * Bootstrap: [7.56k, 14.12k]

**Promotion 1 vs. Promotion 3:**
* Effect Size: +2.73k sales
* P-value: 0.128
* Confidence Interval: [-0.72k, 6.20k]

**Promotion 2 vs. Promotion 3:**
* Effect Size: -8.04k sales (Promotion 2 underperforms)
* P-value: 1.56e-06
* Confidence Interval: [-11.27k, -4.80k]

### Key Findings
* Promotion 1 significantly outperforms Promotion 2 (statistically significant)
* Promotion 1 vs. Promotion 3 shows inconclusive results (confidence interval crosses zero)
* Promotion 3 definitively outperforms Promotion 2 (statistically significant)
* Similar analytical and bootstrap confidence intervals for Promotion 1 vs. 2 strengthen findings' reliability

In [12]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def create_dashboard(df):
    # Create subplot figure
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=('Sales Trends Over Time', 'Sales Distribution by Promotion'))

    # Plot 1: Time series
    weekly_sales = df.groupby(['week', 'Promotion'])['SalesInThousands'].mean().reset_index()
    for promo in sorted(df['Promotion'].unique()):
        promo_data = weekly_sales[weekly_sales['Promotion'] == promo]
        fig.add_trace(
            go.Scatter(x=promo_data['week'], y=promo_data['SalesInThousands'],
                       name=f'Promotion {promo}', mode='lines+markers'),
            row=1, col=1
        )

    # Plot 2: Box plot
    fig.add_trace(
        go.Box(x=df['Promotion'], y=df['SalesInThousands'],
               name='Sales Distribution'),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(height=800, width=1000, showlegend=True,
                      title_text="Fast Food Marketing Campaign Dashboard")

    return fig


dashboard = create_dashboard(df)
dashboard.show()

# Summary

# Fast Food Marketing Campaign A/B Test Analysis

## 1. Goal
The A/B test aimed to determine which of three marketing promotions is most effective for a new menu item at a fast-food chain, measured across multiple locations over four weeks.

## 2. Target Metric
Sales in thousands (SalesInThousands) served as our primary metric. This directly measured the impact on revenue generation.

## 3. Calculations
We've completed:
- Sample size verification (chi-square test showing fair distribution)
 - Promotion 1: 43 locations
 - Promotion 2: 47 locations
 - Promotion 3: 47 locations
 - Chi-square statistic: 0.23, p-value: 0.8898
- Statistical testing (pairwise t-tests)
 - Promotion 1 vs 2: Effect size +10.77k, p-value < 0.001
 - Promotion 1 vs 3: Effect size +2.73k, p-value = 0.128
 - Promotion 2 vs 3: Effect size -8.04k, p-value < 0.001
- Confidence intervals
 - Analytical CI (Promotion 1 vs 2): [7.47k, 14.86k]
 - Bootstrap CI (Promotion 1 vs 2): [7.56k, 14.12k]

## 4. Decision
Based on our statistical analysis:
- Recommend implementing Promotion 1
- It shows significantly higher sales than Promotion 2 (10.77k increase, p<0.001)
- While it performs better than Promotion 3, the difference isn't statistically significant
- The effect is stable across weeks